In [1]:
import glob
import os
import subprocess

import requests
from config import SETTINGS
from geo.Geoserver import Geoserver
from minio import Minio
from minio.error import InvalidResponseError
from tqdm.notebook import tqdm

# Loading GeoTiffs into GeoNode

This notebook illustrates how to load GeoTiffs into GeoServer/GeoNode using GDAL for data processing and the [GeoNode API](https://docs.geonode.org/en/master/devel/index.html). [Geoserver-Rest](https://geoserver-rest.readthedocs.io/en/latest/advanced_uses.html) may also be useful.

## 1. Files to process

The raw mosaics are not well optimised for display online. The first steps are therefore to convert them to Cloud Optimised Geotiffs (COGs), using either lossless or lossy compressions and internal tiling.

In [2]:
# Folder containing image mosaics
data_fold = r"/home/notebook/shared-seabee-ns9879k/all-mosaics"

# Output folder for Cloud Optimised Geotiffs (COGs)
cog_fold = r"/home/notebook/cogs/"

# Set compression type for COGs. If compression = 'LZW', lossless compression
# will be used. If compression = 'JPEG', lossy compression with QUALITY=50
# will be used (this creates much smaller files, but also loses information).
compression = "JPEG"

# Output bit-depth
# NOTE: The JPEG driver only supports 8-bit images, so if compression = 'JPEG'
# the COGs will be converted to 8-bit regardless. If compression = 'LZW', you
# you can choose to create smaller files by converting to 8-bit, but some
# information is lost
conv_8bit = True

# JPEG quality. Only used if compression = 'JPEG'. Must be an integer between
# 1 and 99, where higher means better quality images
jpeg_qual = 90

# Validate user input
assert compression in ("LZW", "JPEG")
if compression == "JPEG":
    print("NOTE: JPEG compression will save only the first three bands.")
    if conv_8bit is False:
        print(
            "JPEG compression requires conversion to 8-bit. Ignoring 'conv_8bit = False'."
        )
    assert jpeg_qual in range(
        1, 100
    ), "'jpeg_qual' must be an integer between 1 and 99."

if (compression == "LZW") and (jpeg_qual is not None):
    print("Using LZW compression. Ignoring 'jpeg_qual = 75'.")

NOTE: JPEG compression will save only the first three bands.


In [3]:
# Get list of files to process
search_path = os.path.join(data_fold, "*.tif")
flist = glob.glob(search_path)

# Print file sizes
print("Raw mosaic sizes:")
for fpath in flist:
    fname = os.path.basename(fpath)
    fsize_mb = os.path.getsize(fpath) / 1e6
    print(f"{fname:<65}: {fsize_mb:.2f} MB")

Raw mosaic sizes:
20220819_1230_RGB_120m_transparent_mosaic_group1.tif             : 4805.99 MB
KelpMap_N_20220819_MS_comp.tif                                   : 4755.82 MB


## 2. Optimising files for display online

The blog post [here](http://blog.cleverelephant.ca/2015/02/geotiff-compression-for-dummies.html) from Paul Ramsey provides a useful staring point for optimising GeoTiffs. Note that, for very large files, the [GeoServer documentation](https://docs.geoserver.org/stable/en/user/tutorials/imagepyramid/imagepyramid.html) recommends building **external image pyramids** rather than **internal overview layers**. Some example bash scripts for building external pyramids are [here](https://www.ianturton.com/tutorials/bluemarble.html). However, loading COGs with internal overviews is easier to automate via the Python API and it's what Paul Ramsey recommends, so we'll test this first.

**Note:** Using `JPEG` compression (`lossless = False` above) will achieve the maximum reduction in file size. This is OK for visualisation online (e.g. for displaying datasets in GeoNode), but the resulting files **must not be used for machine learning**. JPEG compression is lossy and the JPEG format only supports 8-bit data (i.e. other bit depths must be converted). **For the ML we should always work with the original resolution files, saved with `LZW` compression**, which is lossless. However, using `JPEG` compression for visualisation substantially reduces file sizes, giving a smoother user experience.

In [4]:
cog_list = []
for fpath in flist:
    fname = os.path.basename(fpath)
    cog_path = os.path.join(cog_fold, os.path.splitext(fname)[0] + "_cog.tif")
    cog_list.append(cog_path)
    if (compression == "LZW") and (conv_8bit):
        # Use LZW
        cmd = [
            "gdal_translate",
            "-of",
            "COG",
            "-ot",
            "Byte",
            "-co",
            "COMPRESS=LZW",
            "-scale",
            fpath,
            cog_path,
        ]
    elif (compression == "LZW") and (conv_8bit is False):
        cmd = [
            "gdal_translate",
            "-of",
            "COG",
            "-co",
            "COMPRESS=LZW",
            fpath,
            cog_path,
        ]
    else:
        # Convert to 8-bit and compress bands 1 to 3 as JPEG with 50% quality
        cmd = [
            "gdal_translate",
            "-b",
            "1",
            "-b",
            "2",
            "-b",
            "3",
            "-of",
            "COG",
            "-ot",
            "Byte",
            "-co",
            "COMPRESS=JPEG",
            "-co",
            f"QUALITY={jpeg_qual}",
            "-scale",
            fpath,
            cog_path,
        ]
    subprocess.check_call(cmd)

Input file size is 75131, 71027
0...10...20...30...40...50...60...70...80...90...100 - done.


Warning 1: for band 1, nodata value has been clamped to 255, the original value being out of range.
Warning 1: for band 2, nodata value has been clamped to 255, the original value being out of range.
Warning 1: for band 3, nodata value has been clamped to 255, the original value being out of range.


Input file size is 22856, 21624
0...10...20...30...40...50...60...70...80...90...100 - done.


In [5]:
# Get processed file sizes (including all overview layers etc.)
print("COG mosaic sizes:")
for cog_path in cog_list:
    fname = os.path.basename(cog_path)
    fsize_mb = os.path.getsize(cog_path) / 1e6
    print(f"{fname:<65}: {fsize_mb:.2f} MB")

COG mosaic sizes:
20220819_1230_RGB_120m_transparent_mosaic_group1_cog.tif         : 780.32 MB
KelpMap_N_20220819_MS_comp_cog.tif                               : 33.96 MB


The files sizes above using JPEG compression are *much* smaller than the originals (especially since these COGs include internally tiled overviews too).

## 3. Upload COGs to GeoNode

### 3.1. Using the GeoNode API

The simplest way to upload files to GeoNode and have them become available immediately is to use the GeoNode API, as shown below. However, initial testing suggests this is quite slow and may be unreliable for large files. 

In [ ]:
# base_url = "https://geonode.seabee.sigma2.no/api/v2/uploads/upload"
# headers = {"Authorization": f"Bearer {SETTINGS.GEONODE_TOKEN}"}
# for fpath in tqdm(cog_list):
#     fname = os.path.split(fpath)[1]
#     files = [
#         (
#             "base_file",
#             (
#                 fname,
#                 open(fpath, "rb"),
#                 "application/octet-stream",
#             ),
#         ),
#     ]

#     response = requests.request("POST", base_url, headers=headers, files=files)
#     response.raise_for_status()

### 3.2. Using `geoserver-rest`

[geoserver-rest](https://geoserver-rest.readthedocs.io/en/latest/advanced_uses.html) offers direct access to workspaces on GeoServer and it's much nicer to use than the GeoNode API. It also seems faster. However, to get new datasets added to GeoServer to show up in GeoNode, it is necessary to run either `geonode updatelayers` or `python manage.py updatelayers` from the GeoNode shell (sometimes called the Django shell). I *think* this can only be done programatically from within the GeoNode container (i.e. it's not possible to call this from the Hub). However, it looks as though it can be configured via the GeoNode admin panel under

    Home > Management Commands Over HTTP > Management command jobs

This suggests we can either run it manually after new datasets are added, or perhaps set `updatelayers` to run on a schedule. If successful, this may be the best way of getting data into GeoNode.

In [6]:
# Authernticate with GeoServer
geo = Geoserver(
    "https://geonode.seabee.sigma2.no/geoserver",
    username=SETTINGS.GEOSERVER_USER,
    password=SETTINGS.GEOSERVER_PASSWORD,
)

In [7]:
# Upload COGs to GeoServer
workspace = "geonode"

search_path = os.path.join(cog_fold, "*.tif")
flist = glob.glob(search_path)
for fpath in flist:
    fname = os.path.basename(fpath)
    layer_name = os.path.splitext(fname)[0]

    # Add to GeoServer. Note: Will overwrite layer if it exists
    status = geo.create_coveragestore(
        layer_name=layer_name, path=fpath, workspace=workspace
    )
    print(status)

{'coverageStore': {'name': 'KelpMap_N_20220819_MS_comp_cog', 'type': 'GeoTIFF', 'enabled': True, 'workspace': {'name': 'geonode'}, '_default': False, 'dateCreated': '2023-01-16 15:56:21.9 UTC', 'url': 'file:data/geonode/KelpMap_N_20220819_MS_comp_cog/KelpMap_N_20220819_MS_comp_cog.geotiff'}}
{'coverageStore': {'name': '20220819_1230_RGB_120m_transparent_mosaic_group1_cog', 'type': 'GeoTIFF', 'enabled': True, 'workspace': {'name': 'geonode'}, '_default': False, 'dateCreated': '2023-01-16 15:56:25.682 UTC', 'url': 'file:data/geonode/20220819_1230_RGB_120m_transparent_mosaic_group1_cog/20220819_1230_RGB_120m_transparent_mosaic_group1_cog.geotiff'}}


To get the new datasets to appear in GeoNode, login to the GeoNode administration panel and navigate to

    Home > Management Commands Over HTTP > Management command jobs
    
Choose `Add management command job` and set the **Command** to `updatelayers`. Check the **Autostart** box and click **Save**. If you have added a lot of data, the update process may take a while. When it is finished, the status should be updated and the new images datasets be visible in GeoNode.

## 5. Save COGs on MinIO

The final step is to transfer the COGs to MinIO for long-term storage.

In [8]:
# Login to MinIO
sigma2_client = Minio(
    "storage.seabee.sigma2.no",
    access_key=SETTINGS.ACCESS_ID,
    secret_key=SETTINGS.SECRET_KEY,
)

In [9]:
bucket = "all-mosaics"
for fpath in tqdm(cog_list):
    fname = os.path.split(fpath)[1]
    try:
        with open(fpath, "rb") as file_data:
            file_stat = os.stat(fpath)
            sigma2_client.put_object(
                bucket, f"cloud-optimised/{fname}", file_data, file_stat.st_size
            )
    except InvalidResponseError as err:
        print(err)
    os.remove(fpath)

  0%|          | 0/2 [00:00<?, ?it/s]